In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from functions import *
import os
import subprocess
import requests
import re

In [2]:
enzymes = ['dl_endopeptidase', 'ld_carboxypeptidase', 
               'ld_endopeptidase', 'amidase',
               'dd_carboxypeptidase', 'diadenylate_cyclase',
               'muramidase', 'glucosaminidase']

In [ ]:
df = pd.read_csv('../../data/clustering/cluster_maps/dl_endopeptidases.tsv',sep='\t')

df = df.iloc[:, 1:4]

df = df[pd.isna(df[df.columns[2]])]

grouped = df.groupby('dl_endopeptidases-mmseqs_cluster')['dl_endopeptidases-unclustered'].apply(list).reset_index(name='unclustered_list')

display(grouped, len(grouped))

In [37]:
to_download = grouped['unclustered_list'].tolist()

print(to_download)

[['A0A078MK14'], ['A0A1A6AKX1'], ['A0A2U3N4J8'], ['A0A8A0RIX7'], ['A0A8D3WV90'], ['A0A8D3WY21'], ['A0A8F5H2Q7', 'A0A098EI80'], ['A0A8G2HRI5'], ['A0A8J2ZPT7'], ['A0A8S0GDS2'], ['A0A8S0GPC8'], ['A0A8S0YF99'], ['A0A916BQR8', 'A0A1V5WBM4'], ['A0A916CW89'], ['A0A916HNI5'], ['A0A916IAR6'], ['A0A916JZY9', 'W8ZK78', 'A0A919ZDK0'], ['A0A916KA10'], ['A0A916NI20'], ['A0A917ET50', 'I0JQV7'], ['A0A919WN25'], ['A0A926E6B6', 'A0A316LKK8'], ['A0A926HT18', 'J0MY21', 'A0A928K2M9', 'A0A928KYG8', 'A0A6S6XCB1', 'A0A7C7AQD7'], ['A0A927LH80'], ['A0A928LPM8', 'A0A413G4S5', 'A0A926IJ38', 'A0A498CNA2', 'A0A845RI86', 'A0A845SXI2', 'A0A3A9IJD7', 'A0A1Y4MUN7', 'A0A3C1LY33', 'A0A3B9VTI7', 'A0A6N2V297', 'A0A3D0NTZ4', 'A7VT68', 'A0A1C5YFQ8', 'A0A3A9GL57', 'A0A928K5J0', 'A0A143ZRA8', 'A0A1C5LK98', 'A0A1H8CKK5', 'A0A926I4N9', 'A0A926EA13'], ['A0A928NUY4', 'A0A928F840', 'R5PSZ8', 'A0A356GGC1'], ['A0A928PTA1', 'A0A3D5TKR5', 'A0A970U5P3'], ['A0A940NV69', 'A0A3B8NYU2'], ['A0A940ZKM3'], ['A0A942PKR4'], ['A0A949RNE6'], ['A0A

In [15]:
def download_and_check(ids, output_dir, replacements):
    replacement_found = False  # Flag to track if a replacement is found

    for id in ids:
        url = f"https://alphafold.ebi.ac.uk/files/AF-{id}-F1-model_v4.pdb"
        pdb_file_path = os.path.join(output_dir, f"AF-{id}-F1-model_v4.pdb")

        # Download the file
        response = requests.get(url)
        if response.status_code == 200:
            with open(pdb_file_path, 'wb') as file:
                file.write(response.content)

            # Check if "NoSuchKey" is in the file
            with open(pdb_file_path, 'r') as file:
                content = file.read()
                if "NoSuchKey" in content:
                    os.remove(pdb_file_path)  # Delete the file if the string is found
                else:
                    replacements.append(id)
                    replacement_found = True
                    break  # Exit the loop if the string is not found

    if not replacement_found:
        replacements.append('None')  # Append None if no replacement was found




In [40]:
# Directory containing the output files
output_dir = "C:/Users/odesa/Desktop/PDB_test/"
os.makedirs(output_dir, exist_ok=True)

# Loop over each list of IDs
replacements = []
for id_list in to_download:
    download_and_check(id_list, output_dir, replacements)
    # The function will stop processing the current list if "NoSuchKey" is not found in any file

In [41]:
print(replacements)

['A0A078MK14', 'None', 'None', 'None', 'None', 'None', 'A0A098EI80', 'None', 'None', 'None', 'None', 'None', 'A0A1V5WBM4', 'None', 'None', 'None', 'W8ZK78', 'None', 'None', 'I0JQV7', 'None', 'A0A316LKK8', 'J0MY21', 'None', 'A0A413G4S5', 'R5PSZ8', 'A0A3D5TKR5', 'A0A3B8NYU2', 'None', 'None', 'None', 'A0A357WRL2', 'None', 'A0A1Y2MSQ6', 'R6YTV6', 'None', 'A0A6A8FPK7']


In [ ]:
grouped['unclustered_list'] = replacements
print(grouped)

In [51]:
original = pd.read_csv('../../data/clustering/cluster_maps/dl_endopeptidases.tsv',sep='\t')

df = df.iloc[:, 1:4]

In [52]:
merged_df = pd.merge(original, grouped, on='dl_endopeptidases-mmseqs_cluster', how='left')

original['replacements'] = merged_df['unclustered_list']

In [55]:
original['dl_endopeptidases-mmseqs_cluster'] = np.where(original['replacements'].notna(), 
                                                        original['replacements'], original['dl_endopeptidases-mmseqs_cluster'])

In [58]:
# Filter the dataframe based on the condition
filtered_df = grouped[grouped['unclustered_list'] == 'None']

# Get the values from the 'dl_endopeptidases-mmseqs_cluster' column
values = filtered_df['dl_endopeptidases-mmseqs_cluster'].values

# Create a text file and write the values to it
with open('E:/PDBs/dl_endopeptidases_representatives/to_delete.txt', 'w') as file:
    for value in values:
        file.write(str(value) + '\n')


In [41]:
# Needs to be made into a function and optimized. Shoul dimplement multiprocessing for
# downloading the PDBs and checking if they are empty


enzymes = ['dd_endopeptidase']

for enzyme in enzymes:
    
    # read in the cluster map, drop the first column
    df = pd.read_csv('../../data/clustering/cluster_maps/' + enzyme + '.tsv',sep='\t')
    df = df.iloc[:, 1:4]

    # drop the rows where the foldseek cluster is not empty
    no_foldseek = df[pd.isna(df[df.columns[2]])]

    # group by the mmseqs cluster and create a list of the unclustered ids
    grouped = no_foldseek.groupby(f'{enzyme}-mmseqs_cluster')[f'{enzyme}-unclustered'].apply(list).reset_index(name='unclustered_list')

    # create a directory for the PDB files
    to_download = grouped['unclustered_list'].tolist()
    output_dir = f"/Users/odesa/Desktop/PDB_test/{enzyme}"
    os.makedirs(output_dir, exist_ok=True)

    # Loop over each list of IDs
    replacements = []
    for id_list in to_download:
        # download new PDBs, check if empty, creating a list of replacements
        download_and_check(id_list, output_dir, replacements)

    # Replace the 'unclustered_list' column with the list of replacements
    grouped['unclustered_list'] = replacements

    # Merge the original dataframe with the new one on mmseqs cluster
    merged_df = pd.merge(df, grouped, on=f'{enzyme}-mmseqs_cluster', how='left')

    # Create a new column with the list of replacements
    df['replacements'] = merged_df['unclustered_list']

    # Replace the mmseqs cluster with the list of replacements if it is not empty
    df[f'{enzyme}-mmseqs_cluster'] = np.where(df['replacements'].notna(), 
                                                df['replacements'], 
                                                df[f'{enzyme}-mmseqs_cluster'])

    # Filter the dataframe based on the condition
    filtered_df = grouped[grouped['unclustered_list'] == 'None']

    # Get the values from the 'dl_endopeptidases-mmseqs_cluster' column
    values = filtered_df[f'{enzyme}-mmseqs_cluster'].values 

    # Create a text file and write the values to it
    with open(f'/Users/odesa/Desktop/PDB_test/{enzyme}/to_delete.txt', 'w') as file:
        for value in values:
            file.write(str(value) + '\n')

    df.to_csv(f'~/Desktop/PDB_test/{enzyme}_cluster_update.tsv', sep='\t', index=False)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [22]:

for enzyme in enzymes:
    df = pd.read_csv(f'/Users/odesa/Desktop/PDB_test/{enzyme}_cluster_update.tsv', sep='\t')

    # df[f'{enzyme}s-mmseqs_cluster'] = df.apply(lambda row: row[f'{enzyme}s-unclustered'] if pd.isna(row[f'{enzyme}s-mmseqs_cluster']) else row[f'{enzyme}s-mmseqs_cluster'], axis=1)
    
    # df.to_csv(f'C:/Users/odesa/Desktop/{enzyme}_cluster_update.tsv', sep='\t', index=False)
    
    unique_values = df[f'{enzyme}-mmseqs_cluster'].unique().tolist()

    downloaded = []
    with open('/Volumes/PGH-Backup/dd_endopeptidase_clustering/dd_endopeptidases_representatives/all_files.txt', 'r') as file:
        for line in file:
            downloaded.append(line.strip())

    to_remove = [x for x in downloaded if x not in unique_values]
    print(f'{enzyme}: {len(to_remove)} {to_remove}')

    with open('/Volumes/PGH-Backup/dd_endopeptidase_clustering/dd_endopeptidases_representatives/to_remove.txt', 'w') as file:
        for value in to_remove:
            file.write(str(value) + '\n')

dd_endopeptidase: 155 ['A0A061QIV2', 'A0A1F2PE33', 'A0A1T5CWT9', 'A0A1W1UXN8', 'A0A1Y2NTH1', 'A0A2S6X7U8', 'A0A2T4YMR7', 'A0A3N6FDF0', 'A0A5B7V6P8', 'A0A7W4LS90', 'A0A7W7NWJ0', 'A0A7W8GF90', 'A0A7X0G913', 'A0A846UIZ0', 'A0A8A0RL85', 'A0A8A0RM18', 'A0A8A0RPP3', 'A0A8A0RQA4', 'A0A8A0RQE2', 'A0A8B5Y9P4', 'A0A8E1WDG7', 'A0A8E2A495', 'A0A8E2BAA2', 'A0A8F7ZI06', 'A0A8F8UI57', 'A0A8F8Y2Y5', 'A0A8G1UGV3', 'A0A8G2BEL1', 'A0A8G2BG15', 'A0A8G2BHX0', 'A0A8G2CAU3', 'A0A8G2CBI5', 'A0A8G2JCR9', 'A0A8H2K542', 'A0A8H2K898', 'A0A8H2PY24', 'A0A8I0CPR8', 'A0A8I1XTP4', 'A0A8I1YFU9', 'A0A8J7GMD5', 'A0A8J7GRG9', 'A0A8J7KNI4', 'A0A8J7KXG4', 'A0A8J8GDR4', 'A0A8J9XLF8', 'A0A8S0GFC8', 'A0A8T9U3J1', 'A0A916BK31', 'A0A916C815', 'A0A916CDT5', 'A0A916CEK9', 'A0A916CIY5', 'A0A916CLW5', 'A0A916CNC3', 'A0A916CPR8', 'A0A916CRG9', 'A0A916CRP0', 'A0A916CT00', 'A0A916CTX8', 'A0A916CVI7', 'A0A916CWT9', 'A0A916D592', 'A0A916DBM1', 'A0A916DIB5', 'A0A916DPK7', 'A0A916FNN5', 'A0A916FQA4', 'A0A916G0V1', 'A0A916G4Q0', 'A0A916GIL2

In [2]:
enzymes = ['dl_endopeptidase', 'ld_carboxypeptidase', 
               'ld_endopeptidase', 'amidase',
               'dd_carboxypeptidase', 'diadenylate_cyclase',
               'muramidase', 'glucosaminidase']

In [8]:
foldseek_path = '/media/oliver/PGH_Backup/clustering/new_foldseek/'
mmseqs_path = '/media/oliver/PGH_Backup/clustering/maps/'

for enzyme in enzymes:
    df = pd.read_csv(f'{mmseqs_path}{enzyme}_cluster_update.tsv', sep='\t')
    # print(df.head())
    df = df.iloc[:, 0:2]
    # print(df.head())
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    
    df.to_csv(f'{mmseqs_path}{enzyme}_cluster_no_foldseek.tsv', sep='\t', index=False)



In [3]:
foldseek_path = '/media/oliver/PGH_Backup/clustering/new_foldseek/'
mmseqs_path = '/media/oliver/PGH_Backup/clustering/maps/'

for enzyme in enzymes:
    create_maps(f'{mmseqs_path}{enzyme}_cluster_no_foldseek.tsv', f'{foldseek_path}{enzyme}/foldseek_result_{enzyme}_cluster.tsv', f'{enzyme}')

In [ ]:
unique_values = []

for enzyme in enzymes:
    df = pd.read_csv(f'../../data/clustering/cluster_maps/{enzyme}.tsv', sep='\t')
    
    print(f"Original DataFrame shape for {enzyme}: {df.shape}")

    # Get the counts of each value in the target column
    value_counts = df.iloc[:, 3].value_counts()

    # Identify the values that occur only once
    single_occurrence_values = value_counts[value_counts == 1].index

    # Drop rows where the column value is one of those that occur only once
    df = df[~df.iloc[:, 3].isin(single_occurrence_values)]

    df.to_csv(f'../../data/clustering/cluster_maps/{enzyme}.tsv', sep='\t', index=False)

    print(f"Modified DataFrame shape for {enzyme}: {df.shape}")

In [6]:
create_maps('/Volumes/PGH-Backup/foldseek/dd_endopeptidases_clusters.tsv', '/Volumes/PGH-Backup/foldseek/foldseek_result/foldseek_result_cluster.tsv', 'dd_endopeptidase')

In [9]:
# Read the TSV files into DataFrames
df_10_columns = pd.read_csv('../../data/clustering/cluster_maps/combined_clusters.tsv', sep='\t')
df_3_columns = pd.read_csv('../../data/clustering/cluster_maps/dd_endopeptidase.tsv', sep='\t')

# Concatenate the DataFrames along the columns
combined_df = pd.concat([df_10_columns, df_3_columns], axis=1)

# print(combined_df.columns)

# drop the column called Unnamed: 0

combined_df = combined_df.drop('Unnamed: 0', axis=1)

# print(combined_df.columns)

# Save the combined DataFrame to a TSV file
combined_df.to_csv('../../data/clustering/cluster_maps/combined_clusters.tsv', sep='\t', index=False)

/var/folders/sm/rgfxp6g94f18340p2rvqnsc00000gn/T/ipykernel_2092/2862087956.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_10_columns = pd.read_csv('../../data/clustering/cluster_maps/combined_clusters.tsv', sep='\t')


In [42]:
dd_endopeptidase = pd.read_csv('../../data/clustering/cluster_maps/dd_endopeptidase.tsv', sep='\t')

In [25]:
dd_endopeptidase_cluster = pd.read_csv('/Users/odesa/Desktop/PDB_test/dd_endopeptidase_cluster_update.tsv', sep='\t')


display(dd_endopeptidase_cluster)

,dd_endopeptidase-unclustered,dd_endopeptidase-mmseqs_cluster,dd_endopeptidase-foldseek_cluster,replacements
0,A0A0B8XNQ7,A0A0B8XNQ7,A0A0B0HVF0,NaN
1,A0A0G1JBJ2,A0A0G1JBJ2,A4SWU3,NaN
2,A0A0G1Q852,A0A0G1JBJ2,A4SWU3,NaN
3,A0A0J1ISN8,A0A0J1ISN8,A0A0J1ISN8,NaN
4,A0A0P9GGU0,A0A0P9GGU0,A0A840V6T9,NaN
...,...,...,...,...
13857,A0A388Q4W0,A0A388Q4W0,A0A388Q4W0,NaN
13858,A0A399ESD4,A0A399ESD4,A0A1V6AT74,NaN
13859,A0A3D9MWL5,A0A3D9MWL5,A0A1I7HFA3,NaN
13860,A0A3G4VL58,A0A3G4VL58,A0A7W5P753,NaN


In [26]:
dd_endopeptidase_cluster.loc[dd_endopeptidase_cluster['dd_endopeptidase-foldseek_cluster'].isna(), 'dd_endopeptidase-foldseek_cluster'] = dd_endopeptidase_cluster['replacements']

In [27]:
display(dd_endopeptidase_cluster)

,dd_endopeptidase-unclustered,dd_endopeptidase-mmseqs_cluster,dd_endopeptidase-foldseek_cluster,replacements
0,A0A0B8XNQ7,A0A0B8XNQ7,A0A0B0HVF0,NaN
1,A0A0G1JBJ2,A0A0G1JBJ2,A4SWU3,NaN
2,A0A0G1Q852,A0A0G1JBJ2,A4SWU3,NaN
3,A0A0J1ISN8,A0A0J1ISN8,A0A0J1ISN8,NaN
4,A0A0P9GGU0,A0A0P9GGU0,A0A840V6T9,NaN
...,...,...,...,...
13857,A0A388Q4W0,A0A388Q4W0,A0A388Q4W0,NaN
13858,A0A399ESD4,A0A399ESD4,A0A1V6AT74,NaN
13859,A0A3D9MWL5,A0A3D9MWL5,A0A1I7HFA3,NaN
13860,A0A3G4VL58,A0A3G4VL58,A0A7W5P753,NaN


In [30]:
dd_endopeptidase_cluster.drop(columns=['replacements'], inplace=True)

In [31]:
dd_endopeptidase_cluster.to_csv('../../data/clustering/cluster_maps/dd_endopeptidase.tsv', sep = '\t', index=False)

In [38]:
master_map = pd.read_csv('../../data/clustering/cluster_maps/combined_clusters.tsv', sep = '\t')



/var/folders/sm/rgfxp6g94f18340p2rvqnsc00000gn/T/ipykernel_3247/308063967.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,18,19,20,21,22,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  master_map = pd.read_csv('../../data/clustering/cluster_maps/combined_clusters.tsv', sep = '\t')


In [39]:
master_map.drop(columns=['dd_endopeptidase-unclustered', 'dd_endopeptidase-mmseqs_cluster', 'dd_endopeptidase-foldseek_cluster'], inplace=True)

In [43]:
master_map = pd.concat([master_map, dd_endopeptidase_cluster], axis=1)

In [44]:
master_map.to_csv('../../data/clustering/cluster_maps/combined_clusters.tsv', sep = '\t', index=False)